In [58]:
#importa bibliotecas
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import math

In [59]:
#preencher nome da rede geodesica analisada (abaixo os nomes das redes A, B, C, D, E e F, respectivamente)
#alias opcoes = {'gemael147','ghi355gps','nive20obs','gnssrbc21','ghiniv255','kleincorr'}
alias='kleincorr'
aux_filename_36_train="_0-2outs_3-6s_50000itr_train"
aux_filename_69_train="_0-2outs_6-9s_50000itr_train"
aux_filename_36="_0out200000_1-2outs_3-6s_50000itr"
aux_filename_69="_0out200000_1-2outs_6-9s_50000itr"

#from google.colab import drive
#drive.mount('/content/drive')
#caminho="/content/drive/MyDrive/Colab_data/"+alias+"/"+alias+"_"

In [60]:
#importa arquivos para treinamento e validação
caminho="C:/Users/Patricia/Desktop/IP3new/00.treino_rna/"+alias+"_" 
A = np.loadtxt(caminho+"A")
m = A.shape[0]
qtt_exs = m*150000

wAllEL1_pi_sigma_v_36_train = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_36_train).reshape(qtt_exs,1)
ypredEL1_pi_sigma_v_36_train = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_36_train).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_36_train = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_36_train).reshape(qtt_exs,1)
ypredELInf_pp_sig_obs_36_train = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_36_train).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_36_train = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_36_train).reshape(qtt_exs,1)
ypredELInf_pi_sig_obs_36_train = np.loadtxt(caminho+"ypredELInf_pi_sig_obs"+aux_filename_36_train).reshape(qtt_exs,1)
wAllIDS_pp_36_train = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_36_train).reshape(qtt_exs,1)
ypredIDS_pp_36_train = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_36_train).reshape(qtt_exs,1)
wAllSLRTMO_pp_36_train = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_36_train).reshape(qtt_exs,1)
ypredSLRTMO_pp_36_train = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_36_train).reshape(qtt_exs,1)
outs_positions_36_train = np.loadtxt(caminho+"outs_positions"+aux_filename_36_train).reshape(qtt_exs,1)

wAllEL1_pi_sigma_v_69_train = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_69_train).reshape(qtt_exs,1)
ypredEL1_pi_sigma_v_69_train = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_69_train).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_69_train = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_69_train).reshape(qtt_exs,1)
ypredELInf_pp_sig_obs_69_train = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_69_train).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_69_train = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_69_train).reshape(qtt_exs,1)
ypredELInf_pi_sig_obs_69_train = np.loadtxt(caminho+"ypredELInf_pi_sig_obs"+aux_filename_69_train).reshape(qtt_exs,1)
wAllIDS_pp_69_train = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_69_train).reshape(qtt_exs,1)
ypredIDS_pp_69_train = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_69_train).reshape(qtt_exs,1)
wAllSLRTMO_pp_69_train = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_69_train).reshape(qtt_exs,1)
ypredSLRTMO_pp_69_train = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_69_train).reshape(qtt_exs,1)
outs_positions_69_train = np.loadtxt(caminho+"outs_positions"+aux_filename_69_train).reshape(qtt_exs,1)

diag_redund_train = np.loadtxt(caminho+"diag_da_matriz_redund")
diag_redund_train = (np.ones((int(qtt_exs/m),diag_redund_train.shape[0]))*diag_redund_train).reshape(qtt_exs,1)

In [61]:
#importa arquivos para matriz Ev e testes do metaclassificador
caminho="C:/Users/Patricia/Desktop/IP3new/"+alias+"_" 
qtt_exs = m*300000

wAllEL1_pi_sigma_v_36 = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_36).reshape(qtt_exs,1)
ypredEL1_pi_sigma_v_36 = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_36).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_36 = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_36).reshape(qtt_exs,1)
ypredELInf_pp_sig_obs_36 = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_36).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_36 = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_36).reshape(qtt_exs,1)
ypredELInf_pi_sig_obs_36 = np.loadtxt(caminho+"ypredELInf_pi_sig_obs"+aux_filename_36).reshape(qtt_exs,1)
wAllIDS_pp_36 = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_36).reshape(qtt_exs,1)
ypredIDS_pp_36 = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_36).reshape(qtt_exs,1)
wAllSLRTMO_pp_36 = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_36).reshape(qtt_exs,1)
ypredSLRTMO_pp_36 = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_36).reshape(qtt_exs,1)
outs_positions_36 = np.loadtxt(caminho+"outs_positions"+aux_filename_36).reshape(qtt_exs,1)

wAllEL1_pi_sigma_v_69 = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_69).reshape(qtt_exs,1)
ypredEL1_pi_sigma_v_69 = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_69).reshape(qtt_exs,1)
wAllELInf_pp_sig_obs_69 = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_69).reshape(qtt_exs,1)
ypredELInf_pp_sig_obs_69 = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_69).reshape(qtt_exs,1)
wAllELInf_pi_sig_obs_69 = np.loadtxt(caminho+"wAllELInf_pi_sig_obs"+aux_filename_69).reshape(qtt_exs,1)
ypredELInf_pi_sig_obs_69 = np.loadtxt(caminho+"ypredELInf_pi_sig_obs"+aux_filename_69).reshape(qtt_exs,1)
wAllIDS_pp_69 = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_69).reshape(qtt_exs,1)
ypredIDS_pp_69 = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_69).reshape(qtt_exs,1)
wAllSLRTMO_pp_69 = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_69).reshape(qtt_exs,1)
ypredSLRTMO_pp_69 = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_69).reshape(qtt_exs,1)
outs_positions_69 = np.loadtxt(caminho+"outs_positions"+aux_filename_69).reshape(qtt_exs,1)

diag_redund = np.loadtxt(caminho+"diag_da_matriz_redund")
diag_redund = (np.ones((int(qtt_exs/m),diag_redund.shape[0]))*diag_redund).reshape(qtt_exs,1)

In [62]:
#forma conjuntos de treino, validacao e testes
qtd_graphs = 25000 
#qtd de MC cenários de redes geodésicas para cada qtd de outliers e intervalo de magnitude para o treinamento
#a qtd de exemplos de treinamento vai ser isso x5xm, pois entra 0, 1 e 2 outs (sendo 1 e 2 outs para 3-6 e 6-9sigma);


#######train
aux36_train = (ypredIDS_pp_36_train,wAllIDS_pp_36_train,
             ypredEL1_pi_sigma_v_36_train,wAllEL1_pi_sigma_v_36_train,
             ypredELInf_pi_sig_obs_36_train,wAllELInf_pi_sig_obs_36_train,ypredELInf_pp_sig_obs_36_train,wAllELInf_pp_sig_obs_36_train,
             ypredSLRTMO_pp_36_train,wAllSLRTMO_pp_36_train,
              diag_redund_train)
obs_36_train = np.concatenate(aux36_train,axis=1)
label_36_train = outs_positions_36_train

aux69_train = (ypredIDS_pp_69_train,wAllIDS_pp_69_train,
             ypredEL1_pi_sigma_v_69_train,wAllEL1_pi_sigma_v_69_train,
             ypredELInf_pi_sig_obs_69_train,wAllELInf_pi_sig_obs_69_train,ypredELInf_pp_sig_obs_69_train,wAllELInf_pp_sig_obs_69_train,
             ypredSLRTMO_pp_69_train,wAllSLRTMO_pp_69_train,
              diag_redund_train)
obs_69_train = np.concatenate(aux69_train,axis=1) 
label_69_train = outs_positions_69_train

#####teste
aux36 = (ypredIDS_pp_36,wAllIDS_pp_36,
             ypredEL1_pi_sigma_v_36,wAllEL1_pi_sigma_v_36,
             ypredELInf_pi_sig_obs_36,wAllELInf_pi_sig_obs_36,ypredELInf_pp_sig_obs_36,wAllELInf_pp_sig_obs_36,
             ypredSLRTMO_pp_36,wAllSLRTMO_pp_36,
             diag_redund)
X_test36 = np.concatenate(aux36,axis=1)
y_test36 = outs_positions_36

aux69 = (ypredIDS_pp_69,wAllIDS_pp_69,
             ypredEL1_pi_sigma_v_69,wAllEL1_pi_sigma_v_69,
             ypredELInf_pi_sig_obs_69,wAllELInf_pi_sig_obs_69,ypredELInf_pp_sig_obs_69,wAllELInf_pp_sig_obs_69,
             ypredSLRTMO_pp_69,wAllSLRTMO_pp_69,
             diag_redund)
X_test69 = np.concatenate(aux69,axis=1) 
y_test69 = outs_positions_69

######Ev
aux = int(y_test36.shape[0]/m)
y_Ev = np.reshape(y_test36, (aux, m), order='C')
X_Ev = np.reshape(X_test36, (aux, 11*m), order='C')
aux=sum(np.transpose(y_Ev))==0        ##### era y_test (usando o "local" da fç..)
X_Ev = X_Ev[aux,:]
y_Ev = y_Ev[aux,:]
aux = int(X_Ev.shape[0]*m)
X_Ev = np.reshape(X_Ev, (aux, 11), order='C')


X_train36 = np.concatenate((obs_36_train[0:0+m*qtd_graphs,:],obs_36_train[m*50000:m*50000+m*qtd_graphs,:],
                         obs_36_train[m*100000:m*100000+int(m*qtd_graphs),:]),axis=0)
y_train36 = np.concatenate((label_36_train[0:0+m*qtd_graphs,:],label_36_train[m*50000:m*50000+m*qtd_graphs,:],
                         label_36_train[m*100000:m*100000+int(m*qtd_graphs),:]),axis=0)
X_train69 = np.concatenate((obs_69_train[m*50000:m*50000+m*qtd_graphs,:],
                         obs_69_train[m*100000:m*100000+int(m*qtd_graphs),:]),axis=0)
y_train69 = np.concatenate((label_69_train[m*50000:m*50000+m*qtd_graphs,:],
                         label_69_train[m*100000:m*100000+int(m*qtd_graphs),:]),axis=0)
X_val36 = np.concatenate((obs_36_train[m*25000:m*50000,:],obs_36_train[m*75000:m*100000,:],obs_36_train[m*125000:m*150000,:]),axis=0)
X_val69 = np.concatenate((obs_69_train[m*25000:m*50000,:],obs_69_train[m*75000:m*100000,:],obs_69_train[m*125000:m*150000,:]),axis=0)
y_val36 = np.concatenate((label_36_train[m*25000:m*50000,:],label_36_train[m*75000:m*100000,:],label_36_train[m*125000:m*150000,:]),axis=0).flatten('C')
y_val69 = np.concatenate((label_69_train[m*25000:m*50000,:],label_69_train[m*75000:m*100000,:],label_69_train[m*125000:m*150000,:]),axis=0).flatten('C')


VS = 0.8  ###### parte para treino
X_train = np.concatenate((X_train36[0:int(m*qtd_graphs*VS),:],
                          X_train36[int(m*qtd_graphs*1):int(m*qtd_graphs*(1+VS)),:],
                          X_train36[int(m*qtd_graphs*2):int(m*qtd_graphs*(2+VS)),:],
                          X_train69[0:int(m*qtd_graphs*VS),:],
                          X_train69[int(m*qtd_graphs*1):int(m*qtd_graphs*(1+VS)),:],
                          X_train36[int(m*qtd_graphs*VS):int(m*qtd_graphs*1),:],
                          X_train36[int(m*qtd_graphs*(1+VS)):int(m*qtd_graphs*2),:],
                          X_train36[int(m*qtd_graphs*(2+VS)):int(m*qtd_graphs*3),:],
                          X_train69[int(m*qtd_graphs*VS):int(m*qtd_graphs*1),:],
                          X_train69[int(m*qtd_graphs*(1+VS)):int(m*qtd_graphs*2),:]),axis=0)
y_train = np.concatenate((y_train36[0:int(m*qtd_graphs*VS),:],
                          y_train36[int(m*qtd_graphs*1):int(m*qtd_graphs*(1+VS)),:],
                          y_train36[int(m*qtd_graphs*2):int(m*qtd_graphs*(2+VS)),:],
                          y_train69[0:int(m*qtd_graphs*VS),:],
                          y_train69[int(m*qtd_graphs*1):int(m*qtd_graphs*(1+VS)),:],
                          y_train36[int(m*qtd_graphs*VS):int(m*qtd_graphs*1),:],
                          y_train36[int(m*qtd_graphs*(1+VS)):int(m*qtd_graphs*2),:],
                          y_train36[int(m*qtd_graphs*(2+VS)):int(m*qtd_graphs*3),:],
                          y_train69[int(m*qtd_graphs*VS):int(m*qtd_graphs*1),:],
                          y_train69[int(m*qtd_graphs*(1+VS)):int(m*qtd_graphs*2),:]),axis=0)

scaler_train = MinMaxScaler() 
X_train = scaler_train.fit_transform(X_train)
X_val36 = scaler_train.transform(X_val36)
X_val69 = scaler_train.transform(X_val69)
X_test36 = scaler_train.transform(X_test36)
X_test69 = scaler_train.transform(X_test69)
X_Ev = scaler_train.transform(X_Ev)

X_val = np.concatenate((X_val36, X_val69),axis=0)
y_val = np.concatenate((y_val36, y_val69),axis=0).flatten('C')
X_test = np.concatenate((X_test36, X_test69),axis=0)
y_test = np.concatenate((y_test36, y_test69),axis=0).flatten('C')

print(X_train.shape,y_val.shape,X_test36.shape,y_test36.shape,X_Ev.shape,y_Ev.shape)

(4875000, 11) (5850000,) (11700000, 11) (11700000, 1) (7800000, 11) (200000, 39)


In [63]:
#funcao para avaliar desempenho do metaclassificador
def clf_eval(y_pred, y_val):
  if y_pred.ndim < 2 : ## isso eh apenas para o caso de granularidade nas obs..
    aux = int(y_pred.shape[0]/m)
    y_pred = np.reshape(y_pred, (aux, m), order='C')
    y_val = np.reshape(y_val, (aux, m), order='C')
  acc = 100*accuracy_score(y_pred, y_val)
  print('Accuracy: {}'.format(acc))
  #acurácia somente para conjunto que tenha 2 outliers 
  aux=sum(np.transpose(y_val))==2
  acc2=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
  print('Accuracy 2 out: {}'.format(acc2))
  #acurácia somente para conjunto que tenha 1 outliers 
  aux=sum(np.transpose(y_val))==1
  acc1=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
  print('Accuracy 1 out: {}'.format(acc1))
  #acurácia somente para conjunto que tenha 0 outliers 
  aux=sum(np.transpose(y_val))==0
  acc0=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
  print('Accuracy 0 out: {}'.format(acc0))
  acc = (acc2+acc1)/2
  #print('Accuracy: {}'.format(acc))
  acc = [acc, acc1, acc2]
  return acc

In [64]:
#funcao para controle da taxa de falsos positivos do metaclassificador
def fp_control(FP, X_test, y_test):

  probs = np.array(model.predict(X_Ev,batch_size=200*m))
  aux = y_Ev.shape[0]
  #aux = int(y_test.shape[0]/m)
  #y_test_aux = np.reshape(y_test, (aux, m), order='C')
  probs = np.reshape(probs, (aux, m), order='C')
  #aux=sum(np.transpose(y_test_aux))==0
  #maxs = np.sort(np.amax(probs[aux,:], axis=1))
  maxs = np.sort(np.amax(probs, axis=1))
  position = int((1-FP)*maxs.shape[0])
  threshold = maxs[position]

  probs = np.array(model.predict(X_test))
  preds = (np.where(probs[:,:] > threshold, 1, 0))
  #print(preds.shape)
  if y_test.ndim < 2 : ## isso eh apenas para o caso de granularidade nas obs..
    preds = preds[:,0]
  #print(preds.shape)
  label = y_test
  #print(label.shape)
  acc =clf_eval(preds,label)
  return acc

In [65]:
#keras MLP 
hidden1 = math.ceil(X_train.shape[1]*(2/3))
hidden2 = math.ceil(X_train.shape[1]*(1/3))

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", min_delta=0.0001,patience=10, restore_best_weights=True)

# define the keras model
tf.random.set_seed(1234) 
#dropout_rate = 0.1
model = Sequential()
model.add(Dense(hidden1, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(Dense(hidden2, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(units=1, activation='sigmoid'))  

# compile the keras model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy(name="acc")])
#model.summary()

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=200, batch_size=32*m,
          verbose=0,callbacks=[early_stopping],validation_split=1-VS)

#com mudança de valor critico para controlar taxa de FP:
clf=model
print("Validacao:") 
acc36 = fp_control(0.05, X_val36, y_val36) 
acc69 = fp_control(0.05, X_val69, y_val69) 
print("MSR:", (acc36[1] + acc36[2] + acc69[1] + acc69[2])/4)

Validacao:
Accuracy: 82.80133333333333
Accuracy 2 out: 69.1
Accuracy 1 out: 84.416
Accuracy 0 out: 94.88799999999999
Accuracy: 93.55466666666666
Accuracy 2 out: 90.608
Accuracy 1 out: 95.16799999999999
Accuracy 0 out: 94.88799999999999
MSR: 84.823


In [66]:
#keras MLP 
hidden1 = math.ceil(X_train.shape[1]*(2/3))
hidden2 = math.ceil(X_train.shape[1]*(1/3))

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", min_delta=0.0001,patience=10, restore_best_weights=True)

# define the keras model
tf.random.set_seed(1234) 
#dropout_rate = 0.1
model = Sequential()
model.add(Dense(hidden1, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(Dense(hidden2, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(units=1, activation='sigmoid'))  

# compile the keras model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy(name="acc")])
#model.summary()

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=200, batch_size=32*m,
          verbose=0,callbacks=[early_stopping],validation_split=1-VS)

#com mudança de valor critico para controlar taxa de FP:
clf=model
print("Validacao:") 
acc36 = fp_control(0.05, X_val36, y_val36) 
acc69 = fp_control(0.05, X_val69, y_val69) 
print("MSR:", (acc36[1] + acc36[2] + acc69[1] + acc69[2])/4)

Validacao:
Accuracy: 82.83066666666666
Accuracy 2 out: 68.892
Accuracy 1 out: 84.708
Accuracy 0 out: 94.892
Accuracy: 94.268
Accuracy 2 out: 92.052
Accuracy 1 out: 95.86
Accuracy 0 out: 94.892
MSR: 85.378


In [67]:
#keras MLP 
hidden1 = math.ceil(X_train.shape[1]*(2/3))
hidden2 = math.ceil(X_train.shape[1]*(1/2))
hidden3 = math.ceil(X_train.shape[1]*(1/3))

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", min_delta=0.0001,patience=10, restore_best_weights=True)

# define the keras model
tf.random.set_seed(1234) 
#dropout_rate = 0.1
model = Sequential()
model.add(Dense(hidden1, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(Dense(hidden2, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(hidden3, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(units=1, activation='sigmoid'))   

# compile the keras model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy(name="acc")])
#model.summary()

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=200, batch_size=32*m,
          verbose=0,callbacks=[early_stopping],validation_split=1-VS)

#com mudança de valor critico para controlar taxa de FP:
clf=model
print("Validacao:") 
acc36 = fp_control(0.05, X_val36, y_val36) 
acc69 = fp_control(0.05, X_val69, y_val69) 
print("MSR:", (acc36[1] + acc36[2] + acc69[1] + acc69[2])/4)

Validacao:
Accuracy: 82.50800000000001
Accuracy 2 out: 68.56400000000001
Accuracy 1 out: 84.072
Accuracy 0 out: 94.88799999999999
Accuracy: 93.524
Accuracy 2 out: 90.64
Accuracy 1 out: 95.044
Accuracy 0 out: 94.88799999999999
MSR: 84.58


In [68]:
#keras MLP 
hidden1 = math.ceil(X_train.shape[1]*(2/3))
hidden2 = math.ceil(X_train.shape[1]*(1/2))
hidden3 = math.ceil(X_train.shape[1]*(1/3))

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", min_delta=0.0001,patience=10, restore_best_weights=True)

# define the keras model
tf.random.set_seed(1234) 
#dropout_rate = 0.1
model = Sequential()
model.add(Dense(hidden1, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(Dense(hidden2, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(hidden3, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(units=1, activation='sigmoid'))  

# compile the keras model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy(name="acc")])
#model.summary()

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=200, batch_size=32*m,
          verbose=0,callbacks=[early_stopping],validation_split=1-VS)

#com mudança de valor critico para controlar taxa de FP:
clf=model
print("Validacao:") 
acc36 = fp_control(0.05, X_val36, y_val36) 
acc69 = fp_control(0.05, X_val69, y_val69) 
print("MSR:", (acc36[1] + acc36[2] + acc69[1] + acc69[2])/4)

Validacao:
Accuracy: 82.72
Accuracy 2 out: 68.928
Accuracy 1 out: 84.336
Accuracy 0 out: 94.896
Accuracy: 94.028
Accuracy 2 out: 91.792
Accuracy 1 out: 95.396
Accuracy 0 out: 94.896
MSR: 85.113
